# Part-3: Activations, Gradients and BatchNorm

## 1. Introduction

In this part we dive into the practical challenges of training neural networks and introduces powerful techniques like Batch Normalization to make our models train faster and better.

## 2. Setup and Data Loading

In [12]:
import torch
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

### Reading the Names Dataset

Like previous parts we load the names.txt file into a Python list called words.



In [6]:
words = open('names.txt', 'r').read().splitlines() # list of strings
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
len(words)

32033

### Building the Vocabulary

We create our character-to-integer `(stoi)` and integer-to-character `(itos)` mappings. This vocabulary is essential for converting our text data into a numerical format for the network. We also define `vocab_size` which we'll need for the dimensions of our output layer.

In [9]:
chars = sorted(list(set(''.join(words))))# unique characters in the dataset
stoi = {s: i+1 for i, s in enumerate(chars)}# char to int mapping
stoi['.'] = 0# we will use '.' as a special character to represent "end of string"
itos = {i: s for s, i in stoi.items()}# int to char mapping
vocab_size = len(itos)# vocab size
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
print(vocab_size)

27


## 2. Dataset Preparation

### Creating the Dataset (Train/Val/Test Split)

In [13]:
block_size = 3#context length: how many characters do we take to predict the next one?


def build_dataset(words):
  X, Y = [], []# X is the input, Y is the target

  for w in words:
    context = [0] * block_size# initialize the context with 'block_size' zeros
    for ch in w + '.':# we add the special character '.' to the end of each word
      ix = stoi[ch]# get the integer representation of the character
      X.append(context)# append the current context to X
      Y.append(ix)# append the target character to Y
      context = context[1:] + [ix]# slide the context window and add the new character

  X = torch.tensor(X)# convert to tensor
  Y = torch.tensor(Y)# convert to tensor
  print(X.shape, Y.shape)# print the shape of X and Y
  return X, Y# return the dataset


random.seed(42)# for reproducibility
random.shuffle(words)# shuffle the words
n1 = int(0.8*len(words))#80%
n2 = int(0.9*len(words))#90%

Xtr,  Ytr = build_dataset(words[:n1])#80%
Xdev, Ydev = build_dataset(words[n1:n2])#10%
Xte,  Yte = build_dataset(words[n2:])#10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])
